In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import torch
import torch.nn as nn
import cv2
from glasses.nn.models import *
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from glasses.utils.Storage import ForwardModuleStorage, BackwardModuleStorage
from PIL import Image
from io import BytesIO
import requests
from torchvision.transforms import Normalize
import timm

r = requests.get('https://i.insider.com/5df126b679d7570ad2044f3e?width=700&format=jpeg&auto=webp')
im = Image.open(BytesIO(r.content))

In [64]:
from pathlib import Path

model = ResNet.resnet50(pretrained=False)
dummy_input = torch.randn(32,3,224,224)
save_path = Path('/home/zuppif/Desktop/resnet.onnx')

torch.onnx.export(model, dummy_input, str(save_path), verbose=True, input_names=['x'], output_names=['out'])

graph(%x : Float(32:150528, 3:50176, 224:224, 224:1, requires_grad=0, device=cuda:0),
      %decoder.fc.weight : Float(1000:2048, 2048:1, requires_grad=1, device=cuda:0),
      %decoder.fc.bias : Float(1000:1, requires_grad=1, device=cuda:0),
      %497 : Float(64:147, 3:49, 7:7, 7:1, requires_grad=0, device=cuda:0),
      %498 : Float(64:1, requires_grad=0, device=cuda:0),
      %500 : Float(64:64, 64:1, 1:1, 1:1, requires_grad=0, device=cuda:0),
      %501 : Float(64:1, requires_grad=0, device=cuda:0),
      %503 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cuda:0),
      %504 : Float(64:1, requires_grad=0, device=cuda:0),
      %506 : Float(256:64, 64:1, 1:1, 1:1, requires_grad=0, device=cuda:0),
      %507 : Float(256:1, requires_grad=0, device=cuda:0),
      %509 : Float(256:64, 64:1, 1:1, 1:1, requires_grad=0, device=cuda:0),
      %510 : Float(256:1, requires_grad=0, device=cuda:0),
      %512 : Float(64:256, 256:1, 1:1, 1:1, requires_grad=0, device=cuda:0),
      %51

In [65]:
import onnxruntime as ort

ort_session = ort.InferenceSession(str(save_path))

In [66]:
%%timeit
outputs = ort_session.run(None, {'x': np.random.randn(32, 3, 224, 224).astype(np.float32)})

864 ms ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
%%timeit
model(dummy_input)

51.3 ms ± 966 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
from glasses.nn.models.classification.resnet import *
from glasses.nn.att import WithAtt, ECAModule, SpatialSE
from functools import partial


WithAtt(ResNetBottleneckBlock, att=SpatialSE)(32, 64)

# model = ResNet.resnet26(stem=ResNetStemC, block=WithAtt(partial(ResNetBottleneckBlock, shortcut=ResNetShorcutD), se=ECAModule))

In [ ]:
model.encoder.stem[0].conv

In [ ]:
from transfer_weights import clone_model
from benchmark import benchmark

# tmp = ResNet.resnet50d()
# tmp = clone_model(timm.create_model('resnet50d', pretrained=True), tmp)

tmp = timm.create_model('ecaresnet50d', pretrained=True)
benchmark(tmp.eval().cuda(), ResNet.configs['resnet50d'].transform, 128)

In [ ]:
from timm.models.resnet import resnet26d

timm.create_model('resnet26d')

In [ ]:
ResNet.resnet50().summary()

In [ ]:
class NormalizeInverse(Normalize):
    """
    Undoes the normalization and returns the reconstructed images in the input domain.
    """

    def __init__(self, mean, std):
        mean = torch.Tensor(mean)
        std = torch.Tensor(std)
        std_inv = 1 / (std + 1e-7)
        mean_inv = -mean * std_inv
        super().__init__(mean=mean_inv, std=std_inv)

    def __call__(self, tensor):
        return super().__call__(tensor.clone())

In [ ]:
from glasses.interpretability import GradCam, SaliencyMap
cfg = ResNet.configs['resnet18']

transform = cfg.transform
postprocessing = Normalize(-cfg.mean / cfg.std, (1.0 / cfg.std))

x = transform(im).unsqueeze(0)
model = ResNet.resnet18(pretrained=True)

_ = model.interpret(x, using=SaliencyMap()).show()

In [ ]:
def tensor2cam(image, cam):
    image_with_heatmap = image2cam(image.permute(1, 2, 0).cpu().numpy(),
                                   cam.detach().cpu().numpy())

    return torch.from_numpy(image_with_heatmap)


def image2cam(image, cam):
    h, w, c = image.shape

    cam -= np.min(cam)
    cam /= np.max(cam)  # Normalize between 0-1
    cam = cv2.resize(cam, (h, w))
    cam = np.uint8(cam * 255.0)

    img_with_cam = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
    img_with_cam = cv2.cvtColor(img_with_cam, cv2.COLOR_BGR2RGB)
    img_with_cam = img_with_cam + (image * 255)
    img_with_cam /= np.max(img_with_cam)

    return img_with_cam

In [ ]:
model = ResNet.resnet18(pretrained=True).eval()
conv = model.encoder.blocks[-1].block[-1].block.conv2

weights_storage = ForwardModuleStorage(model, [conv])
gradients_storage = BackwardModuleStorage([conv])

cfg = ResNet.configs['resnet18']

transform = cfg.transform

im_tr = transform(im).unsqueeze(0)

x = Variable(im_tr, requires_grad=True)

out = model(x)

cls = torch.argmax(torch.softmax(out, dim=1))

target = torch.zeros(out.size())
target[0][int(cls)] = 1
        
out.backward(gradient=target, retain_graph=True)

In [ ]:
from torchvision.transforms import Normalize

class NormalizeInverse(Normalize):
    """
    Undoes the normalization and returns the reconstructed images in the input domain.
    """

    def __init__(self, mean, std):
        mean = torch.Tensor(mean)
        std = torch.Tensor(std)
        std_inv = 1 / (std + 1e-7)
        mean_inv = -mean * std_inv
        super().__init__(mean=mean_inv, std=std_inv)

    def __call__(self, tensor):
        return super().__call__(tensor.clone())


In [ ]:
weights = weights_storage[conv][0]
grads = gradients_storage[conv][0][0]

post_processing = NormalizeInverse(cfg.mean, cfg.std)

avg_channel_grad = F.adaptive_avg_pool2d(grads.data, 1)
cam = F.relu(torch.sum(weights * avg_channel_grad, dim=1)).squeeze(0)
image_with_heatmap = tensor2cam(post_processing(im_tr).squeeze(0), cam)


In [ ]:
import matplotlib.pyplot as plt


fig = plt.figure()

plt.imshow(image_with_heatmap)

## Get weights

Proposed API

```python
ResNet.resnet50(pretrained=True, dataset='imagenet')

ResNet.pretrained_models
```



In [ ]:
from glasses.utils.PretrainedWeightsProvider import PretrainedWeightsProvider
from pathlib import Path

provider = PretrainedWeightsProvider()
for key in provider.weights_zoo.keys():
    provider[key]

In [ ]:
from glasses.nn.models import *


# model = ResNetXt.resnext101_32x8d(pretrained=True)

VGG.vgg13(pretrained=True).summary()

In [ ]:
import torch
from glasses.nn.models import *
x = torch.rand((1,3,224,224))


FishNet.fishnet99()(x)

In [ ]:
import torch
from glasses.nn.models import *
x = torch.rand((1,3,224,224))


FishNet.fishnet99()(x)

In [ ]:
torch.load('/home/zuppif/Desktop/test.pth')

In [ ]:
from glasses.nn.blocks import Conv2dPad

x = torch.rand((1,32,9,9))

Conv2dPad(32, 64, kernel_size=9, mode = 'auto', padding=0)

In [ ]:
from glasses.nn.models import *

model =  ResNetXt.resnet18(pretrained=True)

In [ ]:
model.encoder.blocks[-1]

In [ ]:
import timm

timm.list_models('vgg')

In [ ]:
import timm
m = timm.create_model('seresnet101', pretrained=True)
m

In [ ]:
from dataclasses import dataclass
from typing import Tuple, List
from pprint import pprint
from benchmark import benchmark
from glasses.utils.ModuleTransfer import ModuleTransfer
from functools import partial
from glasses.nn.models import *
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2
from tqdm.autonotebook import tqdm
import torchvision.transforms as T
import torch.nn as nn
import torch
from PIL import Image
from glasses.utils.PretrainedWeightsProvider import Config


models_weights = {
    'seresnet50': Config(),
#     'efficientnet_b4': Config(resize=380, input_size=380, interpolation='bicubic')
}

zoo_models_mapping = {
    'seresnet50': [partial(timm.create_model, 'seresnet101',pretrained=True), SEResNet.cse_resnet34],
   
#   'efficientnet_b4': [partial(timm.create_model, 'tf_efficientnet_b4', pretrained=True), EfficientNet.efficientnet_b4]
}



# pprint(models_weights)

In [ ]:
model

In [ ]:
model = EfficientNetPytorch.from_pretrained('efficientnet-b0')
model.set_swish(False)
summary(model.cuda(), (3, 224, 224))